In [84]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

from torchvision.models import resnet18, ResNet18_Weights

transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root= './data', train = True, download = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 256, shuffle = True, num_workers = 4)
testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = 256, shuffle = False, num_workers = 4)

# Print class names
classes = trainset.classes
print("Classes in CIFAR-10 dataset:", classes)

#deifne GPU usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

weights = ResNet18_Weights.IMAGENET1K_V1
model = resnet18(weights = weights)


num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10) #10 classes
model = model.to(device)

criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9) #80%
#optimizer = optim.Adam(model.parameters(), lr = 0.001) #78%
optimizer = optim.Adagrad(model.parameters(), lr = 0.01) #80%
print("Model device:", next(model.parameters()).device)


Files already downloaded and verified
Files already downloaded and verified
Classes in CIFAR-10 dataset: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
Model device: cuda:0


In [80]:
num_epochs = 20 #Changing number of epochs
    
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        try:
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 150 == 149:
                print(f'Epoch {epoch + 1}, Batch {i +1}, Loss: {running_loss / 2000:.5f}')
                running_loss = 0.0
        except Exception as e:
            print(f"Error in Epoch {epoch}, Batch {i}: {e}")
            break

Epoch 1, Batch 150, Loss: 0.01311
Epoch 2, Batch 150, Loss: 0.00831
Epoch 3, Batch 150, Loss: 0.00537
Epoch 4, Batch 150, Loss: 0.00407
Epoch 5, Batch 150, Loss: 0.00246
Epoch 6, Batch 150, Loss: 0.00199
Epoch 7, Batch 150, Loss: 0.00190
Epoch 8, Batch 150, Loss: 0.00132
Epoch 9, Batch 150, Loss: 0.00096
Epoch 10, Batch 150, Loss: 0.00094
Epoch 11, Batch 150, Loss: 0.00054
Epoch 12, Batch 150, Loss: 0.00055
Epoch 13, Batch 150, Loss: 0.00048
Epoch 14, Batch 150, Loss: 0.00042
Epoch 15, Batch 150, Loss: 0.00088
Epoch 16, Batch 150, Loss: 0.00038
Epoch 17, Batch 150, Loss: 0.00032
Epoch 18, Batch 150, Loss: 0.00031
Epoch 19, Batch 150, Loss: 0.00052
Epoch 20, Batch 150, Loss: 0.00035


In [82]:
#freeze network
for param in model.parameters():
    param.requires_grad = False

    #unfreeze fuly connected layer
for param in model.fc.parameters():
    param.requires_grad = True
    
#randomize the wights
model.fc.reset_parameters()

#apply optimizer for the last layer
#fine_optimizer = torch.optim.Adam(model.fc.parameters(), lr = 0.1)
fine_optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)
fine_tune_epochs = 3
for epoch in range(fine_tune_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        try:
            inputs, labels = data[0].to(device), data[1].to(device)
            fine_optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            fine_optimizer.step()
            running_loss += loss.item()
            if i % 150 == 149:
                print(f'Epoch {epoch + 1}, Batch {i +1}, Loss: {running_loss / 2000:.4f}')
                running_loss = 0.0
        except Exception as e:
            print(f"Error in Epoch {epoch}, Batch {i}: {e}")
            break

Epoch 1, Batch 100, Loss: 0.0109
Epoch 2, Batch 100, Loss: 0.0006
Epoch 3, Batch 100, Loss: 0.0005


In [85]:
correct = 0
total = 0
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        for label, prediction in zip(labels, predicted):
            if label == prediction:
                correct_pred[classes[label.item()]] += 1
            total_pred[classes[label.item()]] += 1
            
print('Accuracy of the network on the 10 000 test images: %f %%' % (100 * correct / total))

for classname, correct_count in correct_pred.items():
    accuracy = 100* float(correct_count)/ total_pred[classname]
    print(f'Accuracy for: {classname:5s} is {accuracy:0.1f} %')

Accuracy of the network on the 10 000 test images: 10.890000 %
Accuracy for: airplane is 8.0 %
Accuracy for: automobile is 28.3 %
Accuracy for: bird  is 26.7 %
Accuracy for: cat   is 1.7 %
Accuracy for: deer  is 6.1 %
Accuracy for: dog   is 8.5 %
Accuracy for: frog  is 2.8 %
Accuracy for: horse is 2.2 %
Accuracy for: ship  is 11.8 %
Accuracy for: truck is 12.8 %
